In [1]:
import gym
from gym.spaces import Box, Discrete, Tuple, MultiDiscrete
import logging
import random

from ray.rllib.env import MultiAgentEnv
import ray
import os
from ray import tune
from ray.tune import function
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import PPOTrainer, PPOTFPolicy, PPOTorchPolicy
import ray.rllib.agents.ppo as ppo
import numpy as np
import gym
from gym import error
from gym.utils import closer
from PIL import Image
import torch
from pypylon import pylon
import numpy as np
import cv2
from ctypes import *
import glob
import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import gym
from gym import error
from gym.utils import closer
from gym import spaces
from pypylon import pylon
import time
from matplotlib import cm
from stable_baselines.common.schedules import LinearSchedule
import random

Instructions for updating:
non-resource variables are not supported in the long term
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
class Camera_Env(gym.Env):
    def __init__(self, camera, 
                       Width = 417,
                       Height = 404):
        
        self.camera = camera
        self.camera.AcquisitionFrameRateAbs.Value = 10
        self.camera.GainRaw.Value = 36
        self.camera.AcquisitionMode.SetValue('Continuous')
        self.Width = Width
        self.Height = Height
        self.camera.Width.Value = Width
        self.camera.Height.Value = Height
        print(self.camera.Width.GetValue(), self.camera.Height.GetValue())
        # see https://github.com/basler/pypylon/blob/master/samples/opencv.py
        img = pylon.PylonImage()
        self.converter = pylon.ImageFormatConverter()
        # converting to opencv bgr format
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
        
        
        self.range_ex_time = np.concatenate((np.arange(1e3, 45e3, 1000), 
                                             np.arange(50e3, 500e3, 5000), 
                                             np.arange(600e3, 900e3, 50000), 
                                             np.arange(100e4, 500e4, 500000))).astype(np.int64)
        
        number_of_actions = len(self.range_ex_time)
        print(number_of_actions)
        
        self.action_space = Discrete(number_of_actions)
        # image
        self.observation_space = Box(low=0, 
                                    high=255, 
                                    shape=(Height, Width, 3),
                                    dtype = np.uint8)
        
        
    def grab_image(self):
        # see https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
                break
        self.camera.StopGrabbing()
        return image
    
    def basler(self, action):  
        ExposureTimeRaw = int(self.range_ex_time[action])
        self.camera.ExposureTimeRaw.Value = ExposureTimeRaw
        time.sleep(5) 
        
    def check_hist(self, img):
        hist , bin_edges = np.histogram(np.array(img).ravel())
        max_value_bin = bin_edges[np.argmax(hist)]
        if max_value_bin > 50 and max_value_bin < 150:
            pic_ok = True
        else:
            pic_ok = False
        return pic_ok
    
    def reset(self):
        self.camera.ExposureTimeRaw.Value = 20000
    
    def step(self, action):
        self.basler(action)
        img = self.grab_image()
        done = self.check_hist(img)
        if done:
            reward = 1
        else:
            reward = -1
        return img, reward, done, {}
    


In [3]:
class Lens_Env(gym.Env):
    def __init__(self, camera,
                        Height = 404, 
                         Width = 417):
        
        # corning lib
        self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
        #Check if Maxim driver dll is loaded
        eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
        print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        self.Width = Width
        self.Height = Height
        
        self.action_space = MultiDiscrete([69, 99])
        # image
        self.observation_space = Box(low=0, 
                                            high=255, 
                                            shape=(self.Height, self.Width, 3),
                                            dtype = np.uint8)
        
        self.camera_env = Camera_Env(camera)
    
    def lens_movement(self, action):
        x = c_double(action)
        self.lib.Casp_SetFocusVoltage(x)
        time.sleep(2)
        
    def reset(self):
        self.lens_movement(24.0)
        
    def step(self, action):
        self.lens_movement(action)
        img = self.camera_env.grab_image()
        return img, -1, False, {}

In [4]:
class BaslerEnv(MultiAgentEnv):
    def __init__(self, Width = 417,
                       Height = 404,
                       threshold = 85,
                       filepath = r'C:\Users\CIG\Documents\corning_basler_RL\h_Rl'):
        
        self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        self.camera.Open()
        self.basler_env = Camera_Env(self.camera)
        self.corning_env = Lens_Env(self.camera)
        
        # yolo
        self.model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', 
                                         force_reload=True, 
                                         pretrained=True)
        
        self.threshold = threshold
        
        print(True)
        
        
    def reset(self):
        self.basler_env.reset()
        self.corning_env.reset()
        
        self.steps_remaining_at_level = None
        self.num_high_level_steps = 0
        
        self.low_level_agent_id = "low_level_{}".format(self.num_high_level_steps)
        
        return {"high_level_agent": self.basler_env.grab_image(),}
    
    def focus_value(self, img):
        # Calculate the gradient
        sobelx = cv2.Sobel(np.float32(img), cv2.CV_64F, 1 , 0, ksize=5)
        sobely = cv2.Sobel(np.float32(img),cv2.CV_64F, 0, 1, ksize=5)

        abs_sobel_x = cv2.convertScaleAbs(sobelx) # converting back to uint8
        abs_sobel_y = cv2.convertScaleAbs(sobely)
        #print(abs_sobel_x )

        # Combine the two gradients with equal weight
        dst = cv2.addWeighted(abs_sobel_x,0.5,abs_sobel_y,0.5,0)
        #print(dst)

        # Calculate the average gradient for the image
        # I convert it to a numpy array for ease of calculation
        return pl.asarray(dst).mean()
    
    def find_class(self, results, obj = 'car'):
        len_of_class = len(results.pred)
        pred_class = [int(results.pred[0][i].numpy()[5]) for i in range(len(results.pred[0]))]
        #list_of_yolo_classes = list(zip(results.names, range(0, len(results.names))))
        pred_names = [results.names[i] for i in pred_class]
        if obj in pred_names:
            index = pred_names.index(obj)
            return results.pred[0][index].numpy()
        else:
            return np.array([])
        
    
    def step(self, action_dict):
        if "high_level_agent" in action_dict:
            return self._high_level_step(action_dict["high_level_agent"])
        else:
            return self._low_level_step(list(action_dict.values())[0])
        
    
    def _high_level_step(self, action):
        
        obs_camera, reward_camera, self.done_camera, _ = self.basler_env.step(action)
        
        self.steps_remaining_at_level = 10
        self.num_high_level_steps += 1
        
        self.low_level_agent_id = "low_level_{}".format(self.num_high_level_steps)
        
        obs = {self.low_level_agent_id: obs_camera}
        
        rew = {self.low_level_agent_id: 0}
        
        done = {"__all__": False}
        
        return obs, rew, done, {}
    
    def _low_level_step(self, action):
        
        self.steps_remaining_at_level -= 1
        
        action_lens_coarse = 0
        action_lens_fine = action[1]
        if action[0] < 24:
            action_lens_coarse = 24
        else:
            action_lens_coarse = action[0]
        action_lens = float("%s.%s"% (str(action_lens_coarse), str(action_lens_fine)))
        
        obs_lens, reward_lens, done_lens, _ = self.corning_env.step(action_lens)
        obs = {self.low_level_agent_id: obs_lens}
        
        #rew = {self.low_level_agent_id: 1}
        prediction_yolo = self.model_yolo(obs_lens)
        final_result_yolo = self.find_class(prediction_yolo)
        
        done = {"__all__": False}
        if final_result_yolo.size == 0:
            if self.steps_remaining_at_level == 0:
                done[self.low_level_agent_id] = True
                rew["high_level_agent"] = 0
                obs["high_level_agent"] = obs_lens
            else:
                rew = {self.low_level_agent_id: -100 + betta*(10 - self.steps_remaining_at_level)}
        
        else:
            image = Image.fromarray(obs_lens)
            image = image.crop((final_result_yolo[0], 
                             final_result_yolo[1], 
                             final_result_yolo[2], 
                             final_result_yolo[3]))
            foc_value = self.focus_value(image)
            if foc_value > self.threshold:
                done["__all__"] = True
                rew["high_level_agent"] = 1
                obs["high_level_agent"] = obs
            elif self.steps_remaining_at_level == 0 and foc_value < self.threshold:
                done[self.low_level_agent_id] = True
                rew["high_level_agent"] = 0
                obs["high_level_agent"] = obs_lens
            else:
                rew = {self.low_level_agent_id: 1 - 0.001*(foc_value - 85)**2}
                
        im = Image.fromarray(observation)
        filename = self.file_path + "\img_%s_%s_%s.png" % (str(action_lens), 
                                                           str(self.range_ex_time[action_camera]), 
                                                           str(done))
        im.save(filename)
        
        
        return obs, rew, done, {}

In [5]:
from ray.tune.registry import register_env

def env_creator(env_config):
    return  BaslerEnv(env_config)

register_env("BaslerEnv", env_creator)

In [6]:
# Create an instant camera object with the camera device found first.
# camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
# camera.Open()

In [7]:
from ray.rllib.models import ModelCatalog
from ray import tune
from ray.tune import grid_search

In [8]:
from stable_baselines import PPO2


In [9]:
#ModelCatalog.register_custom_model("PPO_2",PPO2)

In [10]:
def policy_mapping_fn(agent_id):
    if agent_id.startswith("low_level_"):
        return "low_level_policy"
    else:
        return "high_level_policy"
    
    
stop = {"training_iteration": 200,
        "timesteps_total": 100,
        "episode_reward_mean": 0.0,}



config = {
    "env": BaslerEnv,
    "num_workers": 0,
    "entropy_coeff": 0.01,
    "multiagent": {
        "policies": {
            
            "high_level_policy": (PPOTFPolicy, 
                                  Box(low=0, 
                                    high=255, 
                                    shape=(404,417, 3),
                                    dtype = np.uint8),
                                  Discrete(148), 
                                  {"gamma": 0.9}),
            
            "low_level_policy": (PPOTFPolicy,
                                 Box(low=0, 
                                    high=255, 
                                    shape=(404,417, 3),
                                    dtype = np.uint8),
                                 MultiDiscrete([69, 99]), 
                                 {"gamma": 0.0}),
            
#             "low_level_policy": (PPOTFPolicy,
#                                  Tuple([
#                                      lens_env.observation_space,
#                                      Discrete(148)
#                                  ]), maze.action_space, {
#                                      "gamma": 0.0
#                                  }),
        },
        "policy_mapping_fn": function(policy_mapping_fn),
    },
    "framework": "tf",#"torch", #if args.torch else "tf",
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
}

from ray.rllib.agents.trainer_template import build_trainer

# <class 'ray.rllib.agents.trainer_template.MyCustomTrainer'>
# MyTrainer = build_trainer(
#     name="MyCustomTrainer",
#     default_policy=MyTFPolicy)

# ray.init()
# tune.run(MyTrainer
trainer = ppo.PPOTrainer(env= "BaslerEnv", config=config)
# results = tune.run("PPO_2", config=config, stop=stop)

417 404
148
eCOMCaspErr: 0 0
417 404
148


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\CIG/.cache\torch\hub\master.zip

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   

Adding autoShape... 
True


ValueError: No default configuration for obs shape [404, 417, 3], you must specify `conv_filters` manually as a model option. Default configurations are only available for inputs of shape [42, 42, K] and [84, 84, K]. You may alternatively want to use a custom model or preprocessor.